In [10]:
from datasets import load_dataset, DatasetDict, Value
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig , BertForMaskedLM
from transformers import GPT2Config, BertConfig
from transformers import Trainer, TrainingArguments
from torch.cuda import empty_cache
import torch
import wandb


wandb.init(name="New test wandb")

if torch.cuda.is_available():
    device = torch.device("cuda")
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0)
else:
    device = torch.device("cpu")


raw_datasets = load_dataset("tiny_shakespeare")

Found cached dataset tiny_shakespeare (/home/nguyenan1/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)
100%|██████████| 3/3 [00:00<00:00, 1532.45it/s]


In [11]:
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["text"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )
training_corpus = get_training_corpus()

In [12]:
from transformers import AutoTokenizer
# old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
old_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)


In [13]:
context_length = 128

def tokenize(element):

    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)

In [14]:
# config = GPT2Config(vocab_size=len(tokenizer),
#     n_ctx=context_length,
#     bos_token_id=tokenizer.bos_token_id,
#     eos_token_id=tokenizer.eos_token_id)
# model = GPT2LMHeadModel(config)

config = BertConfig(vocab_size=len(tokenizer))
model = BertForMaskedLM(config)
model = model.to(device)

In [15]:
# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=True)
data_collator = data_collator.to(device)

args = TrainingArguments(
    output_dir="codeparrot-ds",
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    evaluation_strategy="steps",
    eval_steps=20,
    num_train_epochs=1,
    weight_decay=0.1,
    lr_scheduler_type="cosine",
    learning_rate=5e-3,
    gradient_accumulation_steps=16,
    fp16=True
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)
empty_cache()
trainer.train()

AttributeError: 'DataCollatorForLanguageModeling' object has no attribute 'to'